# 搭建并使用向量数据库

## 前序数据预处理配置

批量处理文件夹中所有文件

In [7]:
import os
from dotenv import load_dotenv, find_dotenv 
# pip install python-dotenv

_ = load_dotenv(find_dotenv())


# 获取folder_path下所有文件路径，储存在file_paths里
file_paths = []
folder_path = '../../data_base/knowledge_path'
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)
print(file_paths)

['../../data_base/knowledge_path/pumkin_book/pumpkin_book.pdf']


In [9]:
from langchain.document_loaders.pdf import PyMuPDFLoader
# from langchain.document_loaders.markdown import UnstructuredMarkdownLoader

# 遍历文件路径并把实例化的loader存放在loaders里
loaders = []

for file_path in file_paths:
    file_type = file_path.split('.')[-1]
    if file_type == 'pdf':
        loaders.append(PyMuPDFLoader(file_path))
    else:
        print(f"Unsupported file type: {file_type} for file {file_path}")

ImportError: cannot import name 'FieldInfoV1' from 'langchain_core.utils.pydantic' (/opt/anaconda3/envs/llm-rag/lib/python3.12/site-packages/langchain_core/utils/pydantic.py)

In [ ]:
# # 下载文件并存储到text
# # 未作数据清洗
# texts = []

# for loader in loaders:
#     texts.extend(loader.load())

# texts[2]

In [ ]:
# 下载文件并存储到text
# 作数据清洗
from langchain.document_loaders.pdf import PyMuPDFLoader
import re
# 下载文件并存储到text
texts = []
for loader in loaders:
    texts.extend(loader.load())   

for text in texts:
    pattern = re.compile(r'[^\u4e00-\u9fff](\n)[^\u4e00-\u9fff]', re.DOTALL)
    text.page_content = re.sub(pattern, lambda match: match.group(0).replace('\n', ''), text.page_content)
    text.page_content = text.page_content.replace('•', '')
    text.page_content = text.page_content.replace(' ', '')

In [ ]:
texts[0].page_content

In [ ]:
# # 下载文件并存储到text
# # 作数据清洗
# from langchain.document_loaders.pdf import PyMuPDFLoader
# import re
# # 下载文件并存储到text
# texts = []
# for loader in loaders:
#     texts.extend(loader.load())   

# for text in texts:
#     text.page_content = re.sub(r'\s*\n\s*', ' ', text.page_content)
#     text.page_content = re.sub(r'[\s•]', '', text.page_content)

载入后的变量类型为`langchain_core.documents.base.Document`, 文档变量类型同样包含两个属性
- `page_content` 包含该文档的内容。
- `meta_data` 为文档相关的描述性数据。

In [ ]:
text = texts[100]
print(f"每一个元素的类型：{type(text)}.", 
    # f"该文档的描述性数据：{text.metadata}", 
    f"查看该文档的内容:\n{text.page_content[0:]}", 
    sep="\n------\n")

In [ ]:
''' 
* RecursiveCharacterTextSplitter 递归字符文本分割
RecursiveCharacterTextSplitter 将按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，
    这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置
RecursiveCharacterTextSplitter需要关注的是4个参数：

* separators - 分隔符字符串数组
* chunk_size - 每个文档的字符数量限制
* chunk_overlap - 两份文档重叠区域的长度
* length_function - 长度计算函数
'''
#导入文本分割器
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
# 知识库中单段文本长度
CHUNK_SIZE = 500

# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50

In [ ]:
# 使用递归字符文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)
# text_splitter.split_text(text.page_content[0:1000])

In [ ]:
split_docs = text_splitter.split_documents(texts)
print(f"切分后的文件数量：{len(split_docs)}")

In [ ]:
print(f"切分后的字符数（可以用来大致评估 token 数）：{sum([len(doc.page_content) for doc in split_docs])}")

In [ ]:
split_docs[2].page_content

## 构建Chroma向量库



Langchain 集成了超过 30 个不同的向量存储库。我们选择 Chroma 是因为它轻量级且数据存储在内存中，这使得它非常容易启动和开始使用。

LangChain 可以直接使用 OpenAI 和百度千帆的 Embedding，同时，我们也可以针对其不支持的 Embedding API 进行自定义，例如，我们可以基于 LangChain 提供的接口，封装一个 zhupuai_embedding，来将智谱的 Embedding API 接入到 LangChain 中。在[附LangChain自定义Embedding封装讲解](./附LangChain自定义Embedding封装讲解.ipynb)中，我们以智谱 Embedding API 为例，介绍了如何将其他 Embedding API 封装到 LangChain
中，欢迎感兴趣的读者阅读。

**注：如果你使用智谱 API，你可以参考讲解内容实现封装代码，也可以直接使用我们已经封装好的代码[zhipuai_embedding.py](./zhipuai_embedding.py)，将该代码同样下载到本 Notebook 的同级目录，就可以直接导入我们封装的函数。在下面的代码 Cell 中，我们默认使用了智谱的 Embedding，将其他两种 Embedding 使用代码以注释的方法呈现，如果你使用的是百度 API 或者 OpenAI API，可以根据情况来使用下方 Cell 中的代码。**

In [ ]:
# 使用 OpenAI Embedding
# from langchain.embeddings.openai import OpenAIEmbeddings
# 使用百度千帆 Embedding
# from langchain.embeddings.baidu_qianfan_endpoint import QianfanEmbeddingsEndpoint
# 使用我们自己封装的智谱 Embedding，需要将封装代码下载到本地使用
from zhipuai_embedding import ZhipuAIEmbeddings 

# 定义 Embeddings
# embedding = OpenAIEmbeddings() 
embedding = ZhipuAIEmbeddings()
# embedding = QianfanEmbeddingsEndpoint()

# 定义持久化路径
persist_directory = '../../data_base/vector_db/chroma'

In [ ]:
# !rm -rf '../../data_base/vector_db/chroma'  # 删除旧的数据库文件（如果文件夹中有文件的话），windows电脑请手动删除

**安装chromadb， 为了避免版本兼容问题，注意安装指定版本**




In [ ]:
# !pip install chroma-hnswlib==0.7.1 chromadb==0.4.3

In [ ]:
from langchain.vectorstores.chroma import Chroma

vectordb = Chroma.from_documents(
    documents=split_docs[:20], # 为了速度，只选择前 20 个切分的 doc 进行生成
    # documents=split_docs,
    embedding=embedding,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)

在此之后，我们要确保通过运行 vectordb.persist 来持久化向量数据库，以便我们在未来的课程中使用。

让我们保存它，以便以后使用！

In [ ]:
# vectordb.persist()
vectordb.PersistentClient(persist_directory)

In [30]:
# import os
# from langchain.vectorstores.chroma import Chroma
# from zhipuai_embedding import ZhipuAIEmbeddings   # 假设这是正确的导入路径

# # 定义持久化目录
# persist_directory = '../../data_base/vector_db/chroma'

# # 创建嵌入模型
# embedding = ZhipuAIEmbeddings()

# try:
#     # 初始化 Chroma 向量数据库
#     vectordb = Chroma.from_documents(
#         documents=split_docs[:20],  # 为了速度，只选择前 20 个切分的 doc 进行生成
#         embedding=embedding,
#         persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
#     )
    
#     # 持久化向量数据库
#     vectordb.persist()
#     print("向量数据库已成功持久化到磁盘。")
# except Exception as e:
#     print(f"持久化过程中发生错误: {e}")

持久化过程中发生错误: type object 'hnswlib.Index' has no attribute 'file_handle_count'


In [29]:
import os
from langchain.vectorstores.chroma import Chroma
from pydantic import BaseModel  # 直接从 pydantic 导入
from zhipuai_embedding import ZhipuAIEmbeddings  # 假设这是正确的导入路径

# 定义持久化目录
persist_directory = '../../data_base/vector_db/chroma'

# 创建嵌入模型
embedding = ZhipuAIEmbeddings()

try:
    # 初始化 Chroma 向量数据库
    vectordb = Chroma.from_documents(
        documents=split_docs[:20],  # 为了速度，只选择前 20 个切分的 doc 进行生成
        embedding=embedding,
        persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
    )
    
    # 持久化向量数据库
    vectordb.persist()
    print("向量数据库已成功持久化到磁盘。")
except Exception as e:
    print(f"持久化过程中发生错误: {e}")

持久化过程中发生错误: type object 'hnswlib.Index' has no attribute 'file_handle_count'


In [ ]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

## 三、向量检索
### 3.1 相似度检索
Chroma的相似度搜索使用的是余弦距离，即：
$$
similarity = cos(A, B) = \frac{A \cdot B}{\parallel A \parallel \parallel B \parallel} = \frac{\sum_1^n a_i b_i}{\sqrt{\sum_1^n a_i^2}\sqrt{\sum_1^n b_i^2}}
$$
其中$a_i$、$b_i$分别是向量$A$、$B$的分量。

当你需要数据库返回严谨的按余弦相似度排序的结果时可以使用`similarity_search`函数。

In [ ]:
question="什么是南瓜书"

In [ ]:
sim_docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(sim_docs)}")

In [ ]:
for i, sim_doc in enumerate(sim_docs):
    print(f"检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

### 3.2 MMR检索
如果只考虑检索出内容的相关性会导致内容过于单一，可能丢失重要信息。

最大边际相关性 (`MMR, Maximum marginal relevance`) 可以帮助我们在保持相关性的同时，增加内容的丰富度。

核心思想是在已经选择了一个相关性高的文档之后，再选择一个与已选文档相关性较低但是信息丰富的文档。这样可以在保持相关性的同时，增加内容的多样性，避免过于单一的结果。

In [ ]:
mmr_docs = vectordb.max_marginal_relevance_search(question,k=3)

In [ ]:
for i, sim_doc in enumerate(mmr_docs):
    print(f"MMR 检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")